## BUILDING A SLAB

From CIS file

In [9]:
from ase.io import read, write
from ase.build import surface

# Step 1: Load the CIF file
bulk_fe = read('mp-13_Fe.cif')

# Step 2: Create a 2-layer slab
slab = surface(bulk_fe, (1, 1, 0), layers=2)

# Step 3: Adjust the surface size to 30 Å × 30 Å
rep_x = int(30 / (bulk_fe.cell[0, 0] * 2 ** 0.5))  # Repeat along x
rep_y = int(30 / bulk_fe.cell[1, 1])              # Repeat along y
slab = slab.repeat((rep_x, rep_y, 1))

# Step 4: Add a vacuum of 15 Å
slab.center(vacuum=15, axis=2)

# Step 5: Save the slab
write('Fe_110.pdb', slab)  # Save as PDB file
write('Fe_110.xyz', slab)  # Save as XYZ file
write('Fe_100.gro', slab)  # Save as gro file

### SLAB DIMENSION

In [42]:
import numpy as np

# Function to read an XYZ file and extract atomic coordinates (in Ångströms)
def read_xyz_file(xyz_file):
    with open(xyz_file, 'r') as file:
        lines = file.readlines()

    atoms = []
    for line in lines[2:]:  # Skip the first two lines
        parts = line.split()
        if len(parts) < 4:
            continue
        x, y, z = map(float, parts[1:4])  # Extract x, y, z coordinates (in Ångströms)
        atoms.append([x, y, z])

    return np.array(atoms), 'Å'  # Return the coordinates in Ångströms

# Function to read a PDB file and extract atomic coordinates (in Ångströms)
def read_pdb_file(pdb_file):
    with open(pdb_file, 'r') as file:
        lines = file.readlines()

    atoms = []
    for line in lines:
        if line.startswith("ATOM") or line.startswith("HETATM"):
            # Extract x, y, z coordinates from columns 31-38, 39-46, 47-54 (in Ångströms)
            x = float(line[30:38].strip())
            y = float(line[38:46].strip())
            z = float(line[46:54].strip())
            atoms.append([x, y, z])

    return np.array(atoms), 'Å'  # Return the coordinates in Ångströms

# Function to read a GRO file and extract atomic coordinates (in nanometers)
def read_gro_file(gro_file):
    with open(gro_file, 'r') as file:
        lines = file.readlines()

    atoms = []
    for line in lines[2:]:  # Skip the first two lines (title and number of atoms)
        parts = line.split()
        if len(parts) < 5:
            continue
        x, y, z = map(float, parts[3:6])  # Extract x, y, z coordinates (in nm)
        atoms.append([x, y, z])

    return np.array(atoms), 'nm'  # Return the coordinates in nanometers

# Function to calculate the dimensions of the slab
def calculate_dimensions(atoms):
    x_min, y_min, z_min = np.min(atoms, axis=0)
    x_max, y_max, z_max = np.max(atoms, axis=0)

    # Calculate the dimensions
    x_dim = x_max - x_min
    y_dim = y_max - y_min
    z_dim = z_max - z_min

    return x_dim, y_dim, z_dim

# Function to convert coordinates to Ångströms
def convert_to_angstroms(dimensions, unit):
    if unit == 'nm':  # If the unit is nm, convert to Ångströms (1 nm = 10 Å)
        return dimensions * 10
    return dimensions  # Already in Ångströms

# Main function
def main():
    file_path = "md_setup/slab.gro"  # Replace with the path to your .xyz, .pdb, or .gro file

    # Determine the file format based on the file extension
    if file_path.endswith(".xyz"):
        atoms, unit = read_xyz_file(file_path)
        file_type = "XYZ"
    elif file_path.endswith(".pdb"):
        atoms, unit = read_pdb_file(file_path)
        file_type = "PDB"
    elif file_path.endswith(".gro"):
        atoms, unit = read_gro_file(file_path)
        file_type = "GRO"
    else:
        print("Unsupported file format")
        return

    x_dim, y_dim, z_dim = calculate_dimensions(atoms)

    # Convert dimensions to Ångströms (if necessary)
    x_dim_angstrom = convert_to_angstroms(x_dim, unit)
    y_dim_angstrom = convert_to_angstroms(y_dim, unit)
    z_dim_angstrom = convert_to_angstroms(z_dim, unit)

    # Convert dimensions to nanometers (1 Å = 0.1 nm)
    x_dim_nanometer = x_dim_angstrom / 10
    y_dim_nanometer = y_dim_angstrom / 10
    z_dim_nanometer = z_dim_angstrom / 10

    # Display the results in both nm and Å, including the input file type
    print(f"File Type: {file_type}")
    print(f"Dimensions of slab:")
    print(f"X = {x_dim_angstrom:.3f} Å = {x_dim_nanometer:.3f} nm")
    print(f"Y = {y_dim_angstrom:.3f} Å = {y_dim_nanometer:.3f} nm")
    print(f"Z = {z_dim_angstrom:.3f} Å = {z_dim_nanometer:.3f} nm")

# Run the script
if __name__ == "__main__":
    main()

File Type: GRO
Dimensions of slab:
X = 28.400 Å = 2.840 nm
Y = 28.400 Å = 2.840 nm
Z = 5.680 Å = 0.568 nm


## THE NUMBER OF ATOMS

In [41]:
def count_atoms(file_path):
    # Get file extension to determine file format
    file_extension = file_path.split('.')[-1].lower()

    # Read the file and count atoms based on format
    with open(file_path, 'r') as file:
        lines = file.readlines()

    if file_extension == 'xyz':
        # .xyz files: The first line contains the number of atoms
        num_atoms = int(lines[0].strip())
    
    elif file_extension == 'pdb':
        # .pdb files: Count atoms by looking for "ATOM" or "HETATM" lines
        num_atoms = sum(1 for line in lines if line.startswith(('ATOM', 'HETATM')))
    
    elif file_extension == 'gro':
        # .gro files: The second line contains the number of atoms
        num_atoms = int(lines[1].strip().split()[0])
    
    else:
        raise ValueError("Unsupported file format. Only .xyz, .pdb, and .gro are supported.")
    
    return num_atoms

# Main function
def main():
    file_path = "md_setup/slab.gro"  # Replace with the path to your .xyz, .pdb, or .gro file
    try:
        num_atoms = count_atoms(file_path)
        print(f"Number of atoms in the file: {num_atoms}")
    except ValueError as e:
        print(e)

# Run the script
if __name__ == "__main__":
    main()

Number of atoms in the file: 563


## SLAB ANALYSIS

- Making the comparison among coordinate files with different types of them.

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Function to analyze and visualize the slab structure
def analyze_slab_structure(file_path):
    if not os.path.isfile(file_path):
        print(f"Error: File {file_path} not found.")
        return None

    file_extension = os.path.splitext(file_path)[-1].lower()

    x_positions, y_positions, z_positions = [], [], []

    if file_extension == '.gro':
        with open(file_path, 'r') as file:
            lines = file.readlines()
        for line in lines[2:-1]:
            try:
                x_coord = float(line.split()[-3]) * 10  # Convert nm to Å
                y_coord = float(line.split()[-2]) * 10  # Convert nm to Å
                z_coord = float(line.split()[-1]) * 10  # Convert nm to Å
                x_positions.append(x_coord)
                y_positions.append(y_coord)
                z_positions.append(z_coord)
            except (ValueError, IndexError):
                continue

    elif file_extension == '.xyz':
        with open(file_path, 'r') as file:
            lines = file.readlines()[2:]
        for line in lines:
            try:
                _, x_coord, y_coord, z_coord = line.split()
                x_positions.append(float(x_coord))
                y_positions.append(float(y_coord))
                z_positions.append(float(z_coord))
            except (ValueError, IndexError):
                continue
    else:
        print("Unsupported file format. Please provide a .gro or .xyz file.")
        return None

    x_positions, y_positions, z_positions = map(np.array, [x_positions, y_positions, z_positions])

    # Slab thickness (Z-axis)
    z_min, z_max = z_positions.min(), z_positions.max()
    slab_thickness_angstrom = z_max - z_min

    # Length along X and Y axes
    x_min, x_max = x_positions.min(), x_positions.max()
    y_min, y_max = y_positions.min(), y_positions.max()
    length_x = x_max - x_min
    length_y = y_max - y_min

    # Total number of atoms
    total_atoms = len(x_positions)

    # Unique positions along X and Y axes
    unique_x_positions = len(np.unique(np.round(x_positions, 3)))
    unique_y_positions = len(np.unique(np.round(y_positions, 3)))

    # Results summary
    results = {
        "Slab Thickness (Å)": slab_thickness_angstrom,
        "Length in X-axis (Å)": length_x,
        "Length in Y-axis (Å)": length_y,
        "Total Number of Atoms": total_atoms,
        "Atoms Aligned on X-axis": unique_x_positions,
        "Atoms Aligned on Y-axis": unique_y_positions,
    }

    return results

# Compare results from GRO and XYZ files
def compare_structures(gro_file, xyz_file):
    gro_results = analyze_slab_structure(gro_file)
    xyz_results = analyze_slab_structure(xyz_file)

    if not gro_results or not xyz_results:
        print("Error in analyzing one or both files.")
        return

    print("\nComparison of GRO and XYZ Files:")
    for key in gro_results:
        print(f"{key}:")
        print(f"  GRO: {gro_results[key]}")
        print(f"  XYZ: {xyz_results[key]}")
        if isinstance(gro_results[key], (int, float)):
            print(f"  Difference: {abs(gro_results[key] - xyz_results[key])}")
        else:
            print("  Difference: N/A")

# Paths to the GRO and XYZ files
gro_file = r"/home/usac/Desktop/newsitt/thesis/md_setup/slab.gro"
xyz_file = r"/home/usac/Desktop/newsitt/thesis/md_setup/slab.xyz"

# Run comparison
compare_structures(gro_file, xyz_file)


Comparison of GRO and XYZ Files:
Slab Thickness (Å):
  GRO: 5.68
  XYZ: 5.680103
  Difference: 0.00010300000000018628
Length in X-axis (Å):
  GRO: 28.4
  XYZ: 28.400517
  Difference: 0.0005170000000020991
Length in Y-axis (Å):
  GRO: 28.4
  XYZ: 28.400517
  Difference: 0.0005170000000020991
Total Number of Atoms:
  GRO: 563
  XYZ: 563
  Difference: 0
Atoms Aligned on X-axis:
  GRO: 21
  XYZ: 21
  Difference: 0
Atoms Aligned on Y-axis:
  GRO: 21
  XYZ: 21
  Difference: 0


# SLAB IN THE BOX

### REFORMAT GRO FILE

In [70]:
def reformat_gro_file(input_file, output_file):
    try:
        with open(input_file, "r") as file:
            lines = file.readlines()

        fixed_lines = []

        # Copy header and atom count lines
        fixed_lines.append(lines[0].strip() + "\n")
        atom_count = int(lines[1].strip())
        fixed_lines.append(f"{atom_count}\n")

        # Reformat atom lines
        for i, line in enumerate(lines[2:-1]):  # Exclude header and box lines
            try:
                # Parse fields assuming possible formatting issues
                resid = int(line[:5].strip())  # Residue number
                resname = line[5:10].strip()  # Residue name
                atomname = line[10:15].strip()  # Atom name
                atomnum = int(line[15:20].strip())  # Atom index
                x = float(line[20:28].strip())  # X-coordinate
                y = float(line[28:36].strip())  # Y-coordinate
                z = float(line[36:44].strip())  # Z-coordinate

                # Reformat line into proper .gro format
                fixed_line = f"{resid:>5}{resname:<5}{atomname:<5}{atomnum:>5}{x:8.3f}{y:8.3f}{z:8.3f}\n"
                fixed_lines.append(fixed_line)
            except ValueError:
                print(f"Skipping malformed line {i + 3}: {line.strip()}")

        # Append the box dimensions
        box_line = lines[-1].strip()
        fixed_lines.append(box_line + "\n")

        # Write the fixed .gro file
        with open(output_file, "w") as outfile:
            outfile.writelines(fixed_lines)

        print(f"Reformatted .gro file written to: {output_file}")

    except Exception as e:
        print(f"Error during reformatting: {e}")


# Specify input and output file paths
input_gro_file = "md_setup/box.gro"  # Replace with your input file
output_gro_file = "md_setup/fixed_box.gro"  # Corrected output file

# Run the function
reformat_gro_file(input_gro_file, output_gro_file)


Reformatted .gro file written to: md_setup/fixed_box.gro


In [84]:
def validate_gro_file(input_file):
    try:
        with open(input_file, "r") as file:
            lines = file.readlines()

        print(f"Validating GRO file: {input_file}")

        errors = []  # To collect all errors
        residues = set()  # To collect unique residues

        # 1. Check Header
        if len(lines) < 3:
            errors.append("The file does not contain enough lines to be a valid .gro file.")
            return False

        header = lines[0].strip()
        if not header:
            errors.append("Missing header line.")
        else:
            print(f"Header: {header}")

        # 2. Check Atom Count Line
        try:
            atom_count = int(lines[1].strip())
            print(f"Declared Atom Count: {atom_count}")
        except ValueError:
            errors.append("The second line (atom count) is not a valid integer.")
            atom_count = None

        # 3. Check Atom Lines
        atom_lines = lines[2:-1]
        if atom_count is not None and len(atom_lines) != atom_count:
            errors.append(
                f"Atom count mismatch: Declared atom count ({atom_count}) "
                f"does not match the number of atom lines ({len(atom_lines)})."
            )

        for i, line in enumerate(atom_lines):
            if len(line.strip()) < 44:
                errors.append(f"Atom line {i + 3} is too short: {line.strip()}")
                continue
            try:
                resid = int(line[:5].strip())
                resname = line[5:10].strip()
                atomname = line[10:15].strip()
                atomnum = int(line[15:20].strip())
                x = float(line[20:28].strip())
                y = float(line[28:36].strip())
                z = float(line[36:44].strip())

                # Add residue to the set
                residues.add(resname)
            except ValueError as e:
                errors.append(f"Atom line {i + 3} is incorrectly formatted: {line.strip()} | Error: {e}")

        if not errors:
            print("All atom lines are correctly formatted.")
        else:
            print(f"Found {len(errors)} issues in atom lines.")

        # Display unique residues
        print(f"Residues found in the GRO file: {', '.join(sorted(residues))}")

        # 4. Check Box Dimensions
        box_line = lines[-1].strip()
        box_values = box_line.split()
        if len(box_values) not in [3, 9]:
            errors.append(f"Box dimensions line is invalid: {box_line}")
        else:
            try:
                box_values = [float(v) for v in box_values]
                print("Box dimensions are valid.")
            except ValueError:
                errors.append(f"Box dimensions line contains non-numeric values: {box_line}")

        # Print all errors
        if errors:
            print("\n### Errors Found in the GRO File ###")
            for error in errors:
                print(f"- {error}")
            return False

        print("\nGRO file format is correct.")
        return True

    except Exception as e:
        print(f"Error during validation: {e}")
        return False


# Specify input file
input_gro_file = "/mnt/newsitt/topic1/slab/slab_bottom.gro"  # Replace with your input .gro file

# Validate the .gro file
is_valid = validate_gro_file(input_gro_file)
if not is_valid:
    print("The GRO file is invalid. Please fix the reported issues.")
else:
    print("The GRO file is valid.")


Validating GRO file: /mnt/newsitt/topic1/slab/slab_bottom.gro
Header: ADSORPTION STUDIES OF CATIONIC SURFACTANTS
Declared Atom Count: 563
Found 563 issues in atom lines.
Residues found in the GRO file: 
Box dimensions are valid.

### Errors Found in the GRO File ###
- Atom line 3 is too short: 0FE     Fe    1   0.080   0.080   0.000
- Atom line 4 is too short: 0FE     Fe    2   0.080   0.080   0.284
- Atom line 5 is too short: 0FE     Fe    3   0.080   0.080   0.568
- Atom line 6 is too short: 0FE     Fe    4   0.080   0.364   0.000
- Atom line 7 is too short: 0FE     Fe    5   0.080   0.364   0.284
- Atom line 8 is too short: 0FE     Fe    6   0.222   0.222   0.142
- Atom line 9 is too short: 0FE     Fe    7   0.080   0.364   0.568
- Atom line 10 is too short: 0FE     Fe    8   0.222   0.222   0.426
- Atom line 11 is too short: 0FE     Fe    9   0.080   0.648   0.000
- Atom line 12 is too short: 0FE     Fe   10   0.080   0.648   0.284
- Atom line 13 is too short: 0FE     Fe   11   0.2

### CHECK THE SLAB COORDINATIONS

## SLAB ADJUSTMENT [info]

In [10]:
import numpy as np

def read_gro_file(filename):
    """
    Reads atomic positions and box dimensions from a .gro file.
    Returns:
        - z_coords: List of z-coordinates of all atoms.
        - box_height: Height of the simulation box (in nm).
    """
    z_coords = []
    box_height = 0.0
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            
            # Number of atoms is indicated on the second line
            atom_count = int(lines[1].strip())
            
            # Extract atom data (lines 3 to atom_count + 2)
            for line in lines[2:2 + atom_count]:
                parts = line.split()
                z_coords.append(float(parts[-1]))  # z-coordinate is the last column
            
            # Extract box dimensions from the last line
            box_line = lines[-1]
            box_height = float(box_line.split()[-1])  # z-dimension is the last value
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except ValueError as ve:
        print(f"Error while reading the file: {ve}")
    
    # Check if z-coordinates were extracted
    if len(z_coords) == 0:
        print("Warning: No valid z-coordinates were found in the file.")
    
    return np.array(z_coords), box_height

def calculate_slab_properties(z_coords, box_height):
    """
    Calculates slab thickness, space above and below the slab, and total box height.
    Args:
        - z_coords: List/array of z-coordinates of all atoms.
        - box_height: Total height of the simulation box.
    Returns:
        - slab_thickness: Thickness of the slab (in nm).
        - space_above: Space above the slab (in nm).
        - space_below: Space below the slab (in nm).
        - adjustment_height: Height needed to move slab to the bottom of the box.
        - box_height: Total box height (in nm).
    """
    if len(z_coords) == 0:
        raise ValueError("z_coords array is empty. Ensure valid data is provided.")
    
    z_min = np.min(z_coords)  # Lowest z-coordinate (bottom of the slab)
    z_max = np.max(z_coords)  # Highest z-coordinate (top of the slab)
    slab_thickness = z_max - z_min
    space_below = z_min
    space_above = box_height - z_max
    adjustment_height = -z_min  # Negative z_min to move the slab down to z = 0
    return slab_thickness, space_above, space_below, adjustment_height, box_height

def main():
    # Input file name
    gro_filename = "/mnt/new_volume/newsitt/topic1/dtac/conf.gro"
    
    # Read atomic positions and box height
    z_coords, box_height = read_gro_file(gro_filename)
    
    try:
        # Calculate slab properties
        slab_thickness, space_above, space_below, adjustment_height, total_height = calculate_slab_properties(z_coords, box_height)
        
        # Print results
        print(f"Slab Thickness: {slab_thickness:.3f} nm")
        print(f"Space Below the Slab: {space_below:.3f} nm")
        print(f"Space Above the Slab: {space_above:.3f} nm")
        print(f"Total Height of the Simulation Box: {total_height:.3f} nm")
        print(f"Height Adjustment to Move Slab to Bottom: {adjustment_height:.3f} nm")
    except ValueError as ve:
        print(f"Error in calculation: {ve}")

if __name__ == "__main__":
    main()


Slab Thickness: 3.857 nm
Space Below the Slab: -0.053 nm
Space Above the Slab: -0.129 nm
Total Height of the Simulation Box: 3.675 nm
Height Adjustment to Move Slab to Bottom: 0.053 nm


In [23]:
import numpy as np
from collections import Counter

def read_gro_file(filename):
    """
    Reads atomic positions, residues, atom names, and box dimensions from a .gro file.
    Returns:
        - z_coords: List of z-coordinates of all atoms.
        - residue_counts: Dictionary with residue names as keys and their atom counts as values.
        - total_atoms: Total number of atoms in the file.
        - box_height: Height of the simulation box (in nm).
    """
    z_coords = []
    residues = []
    box_height = 0.0
    total_atoms = 0
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            
            # Number of atoms is indicated on the second line
            total_atoms = int(lines[1].strip())
            
            # Extract atom data (lines 3 to total_atoms + 2)
            for line in lines[2:2 + total_atoms]:
                parts = line.split()
                
                # Ensure the line has enough parts to extract data
                if len(parts) >= 5:
                    residue_name = parts[0]  # Residue name
                    residues.append(residue_name)
                    z_coords.append(float(parts[-1]))  # z-coordinate is the last column
            
            # Extract box dimensions from the last line
            box_line = lines[-1]
            box_height = float(box_line.split()[-1])  # z-dimension is the last value
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except ValueError as ve:
        print(f"Error while reading the file: {ve}")
    
    # Count the number of atoms for each residue
    residue_counts = Counter(residues)
    
    # Check if z-coordinates were extracted
    if len(z_coords) == 0:
        print("Warning: No valid z-coordinates were found in the file.")
    
    return np.array(z_coords), residue_counts, total_atoms, box_height

def calculate_slab_properties(z_coords, box_height):
    """
    Calculates slab thickness, space above and below the slab, and total box height.
    Args:
        - z_coords: List/array of z-coordinates of all atoms.
        - box_height: Total height of the simulation box.
    Returns:
        - slab_thickness: Thickness of the slab (in nm).
        - space_above: Space above the slab (in nm).
        - space_below: Space below the slab (in nm).
        - adjustment_height: Height needed to move slab to the bottom of the box.
        - box_height: Total box height (in nm).
    """
    if len(z_coords) == 0:
        raise ValueError("z_coords array is empty. Ensure valid data is provided.")
    
    z_min = np.min(z_coords)  # Lowest z-coordinate (bottom of the slab)
    z_max = np.max(z_coords)  # Highest z-coordinate (top of the slab)
    slab_thickness = z_max - z_min
    space_below = z_min
    space_above = box_height - z_max
    adjustment_height = -z_min  # Negative z_min to move the slab down to z = 0
    return slab_thickness, space_above, space_below, adjustment_height, box_height

def main():
    # Input file name
    gro_filename = "/mnt/new_volume/newsitt/topic1/bdac/prep_conf.gro"
    
    # Read atomic positions, residue counts, total atoms, and box height
    z_coords, residue_counts, total_atoms, box_height = read_gro_file(gro_filename)
    
    try:
        # Calculate slab properties
        slab_thickness, space_above, space_below, adjustment_height, total_height = calculate_slab_properties(z_coords, box_height)
        
        # Print results
        print(f"Total Atoms in the GRO File: {total_atoms}")
        print(f"Slab Thickness: {slab_thickness:.3f} nm")
        print(f"Space Below the Slab: {space_below:.3f} nm")
        print(f"Space Above the Slab: {space_above:.3f} nm")
        print(f"Total Height of the Simulation Box: {total_height:.3f} nm")
        print(f"Height Adjustment to Move Slab to Bottom: {adjustment_height:.3f} nm")
        
        # Print residue counts
        if residue_counts:
            print("\nNumber of Atoms for Each Residue:")
            print(f"{'Residue':<10} {'Atom Count':<10}")
            print("-" * 20)
            for residue, count in residue_counts.items():
                print(f"{residue:<10} {count:<10}")
        else:
            print("No residues found in the GRO file.")
    except ValueError as ve:
        print(f"Error in calculation: {ve}")

if __name__ == "__main__":
    main()


Total Atoms in the GRO File: 2442
Slab Thickness: 2.950 nm
Space Below the Slab: 0.060 nm
Space Above the Slab: 1.990 nm
Total Height of the Simulation Box: 5.000 nm
Height Adjustment to Move Slab to Bottom: -0.060 nm

Number of Atoms for Each Residue:
Residue    Atom Count
--------------------
1Fe        1         
2Fe        1         
3Fe        1         
4Fe        1         
5Fe        1         
6Fe        1         
7Fe        1         
8Fe        1         
9Fe        1         
10Fe       1         
11Fe       1         
12Fe       1         
13Fe       1         
14Fe       1         
15Fe       1         
16Fe       1         
17Fe       1         
18Fe       1         
19Fe       1         
20Fe       1         
21Fe       1         
22Fe       1         
23Fe       1         
24Fe       1         
25Fe       1         
26Fe       1         
27Fe       1         
28Fe       1         
29Fe       1         
30Fe       1         
31Fe       1         
32Fe       1         


# FILE PREPARATION

In [12]:
import MDAnalysis as mda
from MDAnalysis.coordinates import GRO

def xyz_to_gro(xyz_file, gro_file):
    u = mda.Universe(xyz_file, format="XYZ")
    with mda.Writer(gro_file, n_atoms=u.atoms.n_atoms) as W:
        W.write(u.atoms)
    print(f"Conversion complete: {gro_file}")

xyz_to_gro("slab.xyz", "slab.gro")

Conversion complete: slab.gro


/home/usac/Desktop/newsitt/thesis/venv/lib/python3.12/site-packages/MDAnalysis/coordinates/GRO.py:421: UserWarning: Supplied AtomGroup was missing the following attributes: resnames. These will be written with default values. Alternatively these can be supplied as keyword arguments.
  warnings.warn(
/home/usac/Desktop/newsitt/thesis/venv/lib/python3.12/site-packages/MDAnalysis/coordinates/GRO.py:479: UserWarning: missing dimension - setting unit cell to zeroed box [0., 0., 0.]
  warnings.warn(wmsg)


## COORDINATE SCREENING

In [12]:
import py3Dmol

# Load the .gro file
with open("/mnt/new_volume/newsitt/topic1/dtac/box.gro") as gro_file:
    gro_data = gro_file.read()

# Create a py3Dmol viewer
viewer = py3Dmol.view()
viewer.addModel(gro_data, "gro")  # Add .gro file data
viewer.setStyle({"sphere": {}})  # Show atoms as spheres
viewer.zoomTo()

# Display the viewer
viewer.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
import py3Dmol

def read_gro_file_with_box_and_units(filename):
    """
    Reads atomic positions and box dimensions from a .gro file.
    Converts units from nanometers to angstroms for py3Dmol visualization.
    Returns:
        - gro_data: The entire contents of the .gro file as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    box_dims = (0.0, 0.0, 0.0)
    try:
        with open(filename, "r") as file:
            gro_data = file.read()
            
            # Extract the last line for box dimensions
            lines = gro_data.strip().split("\n")
            box_line = lines[-1].split()
            if len(box_line) >= 3:
                # Convert box dimensions from nm to Å
                box_dims = tuple(float(dim) * 10 for dim in box_line[:3])
                print(f"Extracted box dimensions (in Å): {box_dims}")
            else:
                print("Box dimensions line is invalid.")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except ValueError as ve:
        print(f"Error while reading the file: {ve}")
    
    return gro_data, box_dims

def visualize_gro_with_box_units(gro_data, box_dims):
    """
    Visualizes the .gro file with simulation box edges using py3Dmol.
    Converts the box dimensions and coordinates to angstroms.
    Args:
        - gro_data: The .gro file contents as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    if box_dims == (0.0, 0.0, 0.0):
        print("Invalid box dimensions. Cannot draw simulation box.")
        return

    viewer = py3Dmol.view(width=800, height=600)
    
    # Add the molecular structure from the .gro file
    viewer.addModel(gro_data, "gro")  # Load .gro file
    viewer.setStyle({"sphere": {"scale": 0.3}})  # Atoms as spheres
    
    # Add the box edges (in Å)
    x_len, y_len, z_len = box_dims
    box_lines = [
        [[0, 0, 0], [x_len, 0, 0]], [[x_len, 0, 0], [x_len, y_len, 0]],
        [[x_len, y_len, 0], [0, y_len, 0]], [[0, y_len, 0], [0, 0, 0]],  # Bottom face
        [[0, 0, z_len], [x_len, 0, z_len]], [[x_len, 0, z_len], [x_len, y_len, z_len]],
        [[x_len, y_len, z_len], [0, y_len, z_len]], [[0, y_len, z_len], [0, 0, z_len]],  # Top face
        [[0, 0, 0], [0, 0, z_len]], [[x_len, 0, 0], [x_len, 0, z_len]],
        [[x_len, y_len, 0], [x_len, y_len, z_len]], [[0, y_len, 0], [0, y_len, z_len]]  # Vertical edges
    ]
    for line in box_lines:
        viewer.addCylinder({"start": {"x": line[0][0], "y": line[0][1], "z": line[0][2]},
                            "end": {"x": line[1][0], "y": line[1][1], "z": line[1][2]},
                            "radius": 0.1, "color": "gray"})

    # Center and zoom the viewer
    viewer.zoomTo()
    
    # Show the viewer
    return viewer.show()

def main():
    # Input .gro file
    gro_filename = "/mnt/new_volume/newsitt/topic1/slab/solvated.gro"
    
    # Read the .gro file and box dimensions
    gro_data, box_dims = read_gro_file_with_box_and_units(gro_filename)
    
    # Visualize the .gro file with the simulation box
    visualize_gro_with_box_units(gro_data, box_dims)

if __name__ == "__main__":
    main()


Extracted box dimensions (in Å): (30.0, 30.0, 30.0)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
from ase.io import read
from ase.visualize import view
from ase import Atoms

def load_gro_with_box(filename):
    """
    Reads a .gro file and extracts atomic positions along with box dimensions.
    Returns:
        - atoms: ASE Atoms object containing atomic positions and box dimensions.
    """
    try:
        with open(filename, "r") as file:
            lines = file.readlines()
            
            # Parse the last line for box dimensions (in nm)
            box_line = lines[-1].split()
            if len(box_line) >= 3:
                box_dims = [float(dim) * 10 for dim in box_line[:3]]  # Convert nm to Å
                print(f"Extracted box dimensions (in Å): {box_dims}")
            else:
                box_dims = None
                print("Box dimensions not found or invalid.")
            
            # Load the atomic structure using ASE
            atoms = read(filename)
            
            # Assign box dimensions to the ASE Atoms object
            if box_dims:
                atoms.set_cell(box_dims)  # Set periodic box
                atoms.set_pbc(True)       # Enable periodic boundary conditions
            
            return atoms
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None

def main():
    # Input .gro file
    gro_filename = "/mnt/newsitt/topic1/slab/conf.gro"
    
    # Load the structure with box dimensions
    atoms = load_gro_with_box(gro_filename)
    
    # Visualize the structure
    if atoms:
        view(atoms)
    else:
        print("Failed to load the .gro file.")

if __name__ == "__main__":
    main()
